In [18]:
import argparse
import datetime
import os
import math
import numpy as np
import pandas as pd
import sys
from keras.utils import plot_model
from sklearn.preprocessing import scale
from timeit import default_timer as timer
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

import importlib
import sys

In [19]:
### set paramter values
#------------------------------------------------------------------------
# general
#------------------------------------------------------------------------
training_ratio = 0.9            # ratio of training data to overall data
input_dim = 520
output_dim = 13                 # number of labels
verbose = 1                     # 0 for turning off logging
seed = 7                        # random number seed for reproducibility
### global constant variables
#------------------------------------------------------------------------
# general
#------------------------------------------------------------------------
INPUT_DIM = 520                 #  number of APs
VERBOSE = 1                     # 0 for turning off logging
#------------------------------------------------------------------------
# stacked auto encoder (sae)
#------------------------------------------------------------------------
# SAE_ACTIVATION = 'tanh'
SAE_ACTIVATION = 'relu'
SAE_BIAS = False
SAE_OPTIMIZER = 'adam'
SAE_LOSS = 'mse'
#------------------------------------------------------------------------
# classifier
#------------------------------------------------------------------------
CLASSIFIER_ACTIVATION = 'relu'
CLASSIFIER_BIAS = False
CLASSIFIER_OPTIMIZER = 'adam'
CLASSIFIER_LOSS = 'binary_crossentropy'
#CLASSIFIER_LOSS = 'categorical_crossentropy'
#------------------------------------------------------------------------
# input files
#------------------------------------------------------------------------
path_train = '../data/UJIIndoorLoc/trainingData2.csv'           # '-110' for the lack of AP.
path_validation = '../data/UJIIndoorLoc/validationData2.csv'    # ditto
#------------------------------------------------------------------------
# output files
#------------------------------------------------------------------------
path_base = '../my_results/'
path_out =  path_base + ''
#path_sae_model = path_base + '_sae_model.hdf5'

batch_size = 10
epochs = 20
#sae_hidden_layers = [256,128,64,128,256]
sae_hidden_layers = [64, 128, 256, 512]
#classifier_hidden_layers = [128,128]
classifier_hidden_layers =  [64,512,128]
dropout = 0.2
N = 8
scaling= 1

random_seed = 0

In [20]:
### initialize random seed generator of numpy
#import os
#os.environ['PYTHONHASHSEED'] = '0'

np.random.seed(random_seed)

In [21]:
import tensorflow as tf

session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1,device_count = {'GPU': 0})
#session_conf = tf.ConfigProto(device_count = {'GPU': 0})
from keras import backend as K
tf.set_random_seed(random_seed)  # initialize random seed generator of tensorflow
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

from keras.layers import Dense, Dropout
from keras.models import Sequential, load_model

In [22]:
train_df = pd.read_csv(path_train, header=0) # pass header=0 to be able to replace existing names
test_df = pd.read_csv(path_validation, header=0)

In [23]:
traintest_df = train_df.append(test_df, ignore_index=True)

In [24]:
train_AP_features = scale(np.asarray(train_df.iloc[:,0:520]).astype(float), axis=1)


In [25]:
 # add a new column
traintest_df['REFPOINT'] = traintest_df.apply(lambda row: str(int(row['SPACEID'])) + str(int(row['RELATIVEPOSITION'])), axis=1)

In [26]:
blds = np.unique(traintest_df[['BUILDINGID']])
flrs = np.unique(traintest_df[['FLOOR']])

In [27]:
x_avg = {}
y_avg = {}
for bld in blds:
    for flr in flrs:
        # map reference points to sequential IDs per building-floor before building labels
        cond = (traintest_df['BUILDINGID']==bld) & (traintest_df['FLOOR']==flr)
        
        _, idx = np.unique(traintest_df.loc[cond, 'REFPOINT'], return_inverse=True) # refer to numpy.unique manual
        traintest_df.loc[cond, 'REFPOINT'] = idx
            
        # calculate the average coordinates of each building/floor
        x_avg[str(bld) + '-' + str(flr)] = np.mean(train_df.loc[cond[:len(train_df)], 'LONGITUDE'])
        y_avg[str(bld) + '-' + str(flr)] = np.mean(train_df.loc[cond[:len(train_df)], 'LATITUDE'])

In [28]:
train_df = pd.concat([train_df,traintest_df['REFPOINT'][:len(train_df)]], axis=1)

In [29]:
len_train = len(train_df) 
len_train

19937

In [30]:
# for consistency in one-hot encoding
blds_all = np.asarray(pd.get_dummies(pd.concat([train_df['BUILDINGID'], test_df['BUILDINGID']])))
flrs_all = np.asarray(pd.get_dummies(pd.concat([train_df['FLOOR'], test_df['FLOOR']]))) # ditto

blds = blds_all[:len_train]
flrs = flrs_all[:len_train]

In [31]:
rfps = np.asarray(pd.get_dummies(train_df['REFPOINT']))
train_labels = np.concatenate((blds, flrs, rfps), axis=1)
OUTPUT_DIM = train_labels.shape[1]

In [32]:
len(rfps[0])

110

In [33]:
# split the training set into training and validation sets; 

# we will use the validation set at a testing set.
train_val_split = np.full((len(train_AP_features)), True)
train_val_split[int(len(train_AP_features)*training_ratio):] = False

In [34]:

x_train = train_AP_features[train_val_split]
y_train = train_labels[train_val_split]
x_val = train_AP_features[~train_val_split]
y_val = train_labels[~train_val_split]

In [35]:
# create a model based on stacked autoencoder (SAE)
model = Sequential()
model.add(Dense(sae_hidden_layers[0], input_dim=INPUT_DIM, activation=SAE_ACTIVATION, use_bias=SAE_BIAS))
for units in sae_hidden_layers[1:]:
    model.add(Dense(units, activation=SAE_ACTIVATION, use_bias=SAE_BIAS))  
model.add(Dense(INPUT_DIM, activation=SAE_ACTIVATION, use_bias=SAE_BIAS))
model.compile(optimizer=SAE_OPTIMIZER, loss=SAE_LOSS)

# train the model
#model.fit(x_train, x_train, batch_size=batch_size, epochs=epochs, verbose=VERBOSE,shuffle=False)
model.fit(x_train, x_train, batch_size=batch_size, epochs=epochs, verbose=VERBOSE)




Epoch 1/20
17943/17943 [==============================] - 11s 585us/step - loss: 0.5396
Epoch 2/20
17943/17943 [==============================] - 10s 570us/step - loss: 0.4750
Epoch 3/20
17943/17943 [==============================] - 10s 571us/step - loss: 0.4630
Epoch 4/20
17943/17943 [==============================] - 10s 576us/step - loss: 0.4545
Epoch 5/20
17943/17943 [==============================] - 11s 586us/step - loss: 0.4512
Epoch 6/20
17943/17943 [==============================] - 10s 576us/step - loss: 0.4492
Epoch 7/20
17943/17943 [==============================] - 10s 574us/step - loss: 0.4479
Epoch 8/20
17943/17943 [==============================] - 10s 567us/step - loss: 0.4397
Epoch 9/20
17943/17943 [==============================] - 10s 579us/step - loss: 0.4309
Epoch 10/20
17943/17943 [==============================] - 11s 586us/step - loss: 0.4280
Epoch 11/20
17943/17943 [==============================] - 10s 575us/step - loss: 0.4259
Epoch 12/20
17943/17943 [=====

In [36]:
# remove the decoder part
num_to_remove = (len(sae_hidden_layers) + 1) // 2
for i in range(num_to_remove):
    model.pop()
    
### build and train a complete model with the trained SAE encoder and a new classifier
model.add(Dropout(dropout))
for units in classifier_hidden_layers:
    model.add(Dense(units, activation=CLASSIFIER_ACTIVATION, use_bias=CLASSIFIER_BIAS))
    model.add(Dropout(dropout))
model.add(Dense(OUTPUT_DIM, activation='sigmoid', use_bias=CLASSIFIER_BIAS)) # 'sigmoid' for multi-label classification
model.compile(optimizer=CLASSIFIER_OPTIMIZER, loss=CLASSIFIER_LOSS, metrics=['accuracy'])


In [37]:
model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=batch_size, epochs=epochs, verbose=VERBOSE)

Train on 17943 samples, validate on 1994 samples
Epoch 1/20
17943/17943 [==============================] - 6s 346us/step - loss: 0.0629 - acc: 0.9864 - val_loss: 0.0546 - val_acc: 0.9901
Epoch 2/20
17943/17943 [==============================] - 6s 327us/step - loss: 0.0441 - acc: 0.9905 - val_loss: 0.0535 - val_acc: 0.9895
Epoch 3/20
17943/17943 [==============================] - 6s 335us/step - loss: 0.0403 - acc: 0.9907 - val_loss: 0.0533 - val_acc: 0.9897
Epoch 4/20
17943/17943 [==============================] - 6s 325us/step - loss: 0.0378 - acc: 0.9908 - val_loss: 0.0504 - val_acc: 0.9900
Epoch 5/20
17943/17943 [==============================] - 6s 326us/step - loss: 0.0363 - acc: 0.9908 - val_loss: 0.0557 - val_acc: 0.9891
Epoch 6/20
17943/17943 [==============================] - 6s 328us/step - loss: 0.0354 - acc: 0.9908 - val_loss: 0.0547 - val_acc: 0.9892
Epoch 7/20
17943/17943 [==============================] - 6s 325us/step - loss: 0.0350 - acc: 0.9908 - val_loss: 0.0604 - v

In [38]:
# turn the given validation set into a testing set
#test_AP_features = scale(np.asarray(test_df.iloc[:,0:520]).astype(float), axis=1) # convert integer to float and scale jointly (axis=1)
#x_test_utm = np.asarray(test_df['LONGITUDE'])
#y_test_utm = np.asarray(test_df['LATITUDE'])
#blds = blds_all[len_train:]
#flrs = flrs_all[len_train:]

test_AP_features = x_val
x_test_utm = train_df.loc[~train_val_split, 'LONGITUDE'].values
y_test_utm = train_df.loc[~train_val_split, 'LATITUDE'].values
blds = y_val[:,:3]
flrs =y_val[:,3:8]
rps = y_val[:, 8:118]

In [39]:
### evaluate the model
# calculate the accuracy of building and floor estimation
preds = model.predict(test_AP_features, batch_size=batch_size)
n_preds = preds.shape[0]


np.argmax(blds, axis=1)

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [40]:
# blds_results = (np.equal(np.argmax(test_labels[:, :3], axis=1), np.argmax(preds[:, :3], axis=1))).astype(int)
blds_results = (np.equal(np.argmax(blds, axis=1), np.argmax(preds[:, :3], axis=1))).astype(int)
acc_bld = blds_results.mean()
acc_bld

0.99297893681043126

In [41]:
flrs_results = (np.equal(np.argmax(flrs, axis=1), np.argmax(preds[:, 3:8], axis=1))).astype(int)
acc_flr = flrs_results.mean()
acc_bf = (blds_results*flrs_results).mean()
acc_bf

0.8771313941825476

In [42]:
rfps

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [43]:
np.argmax(preds[:, 8:], axis=1)

array([36, 36, 36, ...,  3,  0,  0], dtype=int64)

In [44]:
rps_results = (np.equal(np.argmax(rps, axis=1), np.argmax(preds[:, 8:], axis=1))).astype(int)
acc_rps = rps_results.mean()
acc_rps

0.20260782347041123

In [45]:
acc = (blds_results*flrs_results*rps_results).mean()
acc

0.20160481444332998

In [46]:
# calculate positioning error when building and floor are correctly estimated
mask = np.logical_and(blds_results, flrs_results) # mask index array for correct location of building and floor

In [47]:
x_test_utm = x_test_utm[mask]
y_test_utm = y_test_utm[mask]
blds = blds[mask]
flrs = flrs[mask]
rfps = (preds[mask])[:, 8:118]

In [48]:
# number of correct building and floor location
n_success = len(blds)   
n_success

1749

In [49]:
n_loc_failure = 0
sum_pos_err = 0.0
sum_pos_err_weighted = 0.0
idxs = np.argpartition(rfps, -N)[:, -N:]  # (unsorted) indexes of up to N nearest neighbors
threshold = scaling*np.amax(rfps, axis=1)

In [50]:
blds.shape

(1749, 3)

In [51]:
for i in range(n_success):
    xs = []
    ys = []
    ws = []
    for j in idxs[i]:
        rfp = np.zeros(110)
        rfp[j] = 1
        rows = np.where((train_labels == np.concatenate((blds[i], flrs[i], rfp))).all(axis=1)) # tuple of row indexes
        if rows[0].size > 0:
            if rfps[i][j] >= threshold[i]:
                xs.append(train_df.loc[train_df.index[rows[0][0]], 'LONGITUDE'])
                ys.append(train_df.loc[train_df.index[rows[0][0]], 'LATITUDE'])
                ws.append(rfps[i][j])
    if len(xs) > 0:
        sum_pos_err += math.sqrt((np.mean(xs)-x_test_utm[i])**2 + (np.mean(ys)-y_test_utm[i])**2)
        sum_pos_err_weighted += math.sqrt((np.average(xs, weights=ws)-x_test_utm[i])**2 + (np.average(ys, weights=ws)-y_test_utm[i])**2)
    else:
        n_loc_failure += 1
        key = str(np.argmax(blds[i])) + '-' + str(np.argmax(flrs[i]))
        pos_err = math.sqrt((x_avg[key]-x_test_utm[i])**2 + (y_avg[key]-y_test_utm[i])**2)
        sum_pos_err += pos_err
        sum_pos_err_weighted += pos_err

In [52]:
 # mean_pos_err = sum_pos_err / (n_success - n_loc_failure)
mean_pos_err = sum_pos_err / n_success
# mean_pos_err_weighted = sum_pos_err_weighted / (n_success - n_loc_failure)
mean_pos_err_weighted = sum_pos_err_weighted / n_success
loc_failure = n_loc_failure / n_success # rate of location estimation failure given that building and floor are correctly located

In [53]:
mean_pos_err_weighted

9.203217477211561

In [54]:
### print out final results
now = datetime.datetime.now
path_out = "../my_results/"
#path_out += "_" + now.strftime("%Y%m%d-%H%M%S") + ".txt"
path_out += "[SAE" + str(sae_hidden_layers) + "] "
path_out += "[Class" +str(classifier_hidden_layers) +  "] "
path_out += "[DropOut" + str(dropout)+"] "
path_out += "[PE" + str(round(mean_pos_err,2))+"] "
path_out += "[PEW" + str(round(mean_pos_err_weighted,2))+"] "
path_out += ".txt"
f = open(path_out, 'w')
f.write("#+STARTUP: showall\n")  # unfold everything when opening
f.write("* System parameters\n")
f.write("  - Numpy random number seed: %d\n" % random_seed)
f.write("  - Ratio of training data to overall data: %.2f\n" % training_ratio)
f.write("  - Number of epochs: %d\n" % epochs)
f.write("  - Batch size: %d\n" % batch_size)
f.write("  - Number of neighbours: %d\n" % N)
f.write("  - Scaling factor for threshold: %.2f\n" % scaling)
f.write("  - SAE hidden layers: %d" % sae_hidden_layers[0])
for units in sae_hidden_layers[1:]:
    f.write("-%d" % units)
f.write("\n")
f.write("  - SAE activation: %s\n" % SAE_ACTIVATION)
f.write("  - SAE bias: %s\n" % SAE_BIAS)
f.write("  - SAE optimizer: %s\n" % SAE_OPTIMIZER)
f.write("  - SAE loss: %s\n" % SAE_LOSS)
f.write("  - Classifier hidden layers: ")
if classifier_hidden_layers == '':
    f.write("N/A\n")
else:
    f.write("%d" % classifier_hidden_layers[0])
    for units in classifier_hidden_layers[1:]:
        f.write("-%d" % units)
    f.write("\n")
f.write("  - Classifier hidden layer activation: %s\n" % CLASSIFIER_ACTIVATION)
f.write("  - Classifier bias: %s\n" % CLASSIFIER_BIAS)
f.write("  - Classifier optimizer: %s\n" % CLASSIFIER_OPTIMIZER)
f.write("  - Classifier loss: %s\n" % CLASSIFIER_LOSS)
f.write("  - Classifier dropout rate: %.2f\n" % dropout)
# f.write("  - Classifier class weight for buildings: %.2f\n" % building_weight)
# f.write("  - Classifier class weight for floors: %.2f\n" % floor_weight)
f.write("* Performance\n")
f.write("  - Accuracy (building): %e\n" % acc_bld)
f.write("  - Accuracy (floor): %e\n" % acc_flr)
f.write("  - Accuracy (building-floor): %e\n" % acc_bf)
f.write("  - Location estimation failure rate (given the correct building/floor): %e\n" % loc_failure)
f.write("  - Positioning error (meter): %e\n" % mean_pos_err)
f.write("  - Positioning error (weighted; meter): %e\n" % mean_pos_err_weighted)
f.close()


In [55]:
mean_pos_err_weighted

9.203217477211561

In [56]:
#9.277102567499114
#9.826676547695087
#9.836135151319597
#9.875814741893349

In [57]:
#9.516800349169966
#9.516800349169966
#9.516800349169966
#9.516800349169966
#9.516800349169966

In [58]:
#9.402408856112912
#9.402408856112912
#9.402408856112912
#8.714254619707846
#8.714254619707846